---


## CARGA DE LIBRERÍA Y DATOS

---

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, classification_report
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import chi2, f_oneway,chi2_contingency
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import FunctionTransformer, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Cupido_IA_project/train.csv')
test = pd.read_csv("/content/drive/MyDrive/Cupido_IA_project/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/Cupido_IA_project/sample_submission.csv")

---

## DEFINICIÓN PREPROCESADO

---

In [6]:
cols_to_int = ['age', 'sex', 'cp', 'restecg']

rename_dict = {
    "age": "edad",
    "sex": "sexo",
    "cp": "tipo_dolor_pecho",
    "trestbps": "tension_en_descanso",
    "chol": "colesterol",
    "fbs": "azucar",
    "restecg": "electro_en_descanso",
    "thalach": "latidos_por_minuto",
    "exang": "dolor_pecho_con_ejercicio",
    "oldpeak": "cambio_linea_corazon_ejercicio",
    "slope": "forma_linea_corazon_ejercicio",
    "ca": "num_venas_grandes",
    "thal": "estado_corazon_thal"
}

cols_a_clippear = [
    'tension_en_descanso', 'colesterol',
    'latidos_por_minuto', 'cambio_linea_corazon_ejercicio'
]

categorical_cols_to_round = [
    'num_venas_grandes', 'estado_corazon_thal', 'sexo',
    'tipo_dolor_pecho', 'dolor_pecho_con_ejercicio',
    'azucar', 'forma_linea_corazon_ejercicio', 'electro_en_descanso'
]

---

FUNCIONES DE PREPROCESADO

---

In [7]:
def limpieza_inicial(df):
    """
    Realiza conversiones de tipos, renombres y limpieza básica de errores (-9).
    Se puede aplicar a todo el dataset antes del split.
    """
    df = df.copy()

    for col in cols_to_int:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

    object_cols = df.select_dtypes(include=['object']).columns
    for col in object_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df.rename(columns=rename_dict)
    df.replace([-9, -9.0], np.nan, inplace=True)

    return df

In [8]:
def limpiar_ceros_fisiologicos(X):
    X = X.copy()
    cols_imposibles_con_cero = ['tension_en_descanso', 'colesterol']
    for col in cols_imposibles_con_cero:
        if col in X.columns:
            X[col] = X[col].replace({0: np.nan, 0.0: np.nan})
    return X

In [9]:
def clipear_outliers(X):
    X = X.copy()
    for col in cols_a_clippear:
        if col in X.columns:
            p1 = X[col].quantile(0.01)
            p99 = X[col].quantile(0.99)
            X[col] = X[col].clip(lower=p1, upper=p99)
    return X

In [10]:
def crear_flags_mnar(df):
    """
    ACTUALIZADO (Estrategia Pruning):
    Solo creamos flag para 'num_venas_grandes'.
    'estado_corazon_thal_is_missing' se considera ruido (Grupo 1) y no se genera.
    """
    df_new = df.copy()
    # Solo venas, thal is missing se elimina por V=0.040
    cols_mnar = ['num_venas_grandes']
    for col in cols_mnar:
        if col in df_new.columns:
            df_new[f'{col}_is_missing'] = df_new[col].isna().astype(int)
    return df_new

In [11]:
class RobustKNNImputerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        self.scaler = RobustScaler()
        self.imputer = KNNImputer(n_neighbors=n_neighbors, weights='distance')
        self.feature_names_in_ = None

    def fit(self, X, y=None):
        self.feature_names_in_ = X.columns if hasattr(X, 'columns') else [f"feat_{i}" for i in range(X.shape[1])]
        X_scaled = self.scaler.fit_transform(X)
        self.imputer.fit(X_scaled)
        return self

    def transform(self, X):
        X_scaled = self.scaler.transform(X)
        X_imputed_scaled = self.imputer.transform(X_scaled)
        X_imputed = self.scaler.inverse_transform(X_imputed_scaled)
        return pd.DataFrame(X_imputed, columns=self.feature_names_in_, index=X.index)

    def set_output(self, *, transform=None):
        return self

In [12]:
def redondear_imputaciones(X):
    X = X.copy()
    for col in categorical_cols_to_round:
        if col in X.columns:
            X[col] = X[col].round()
    return X

In [13]:
def aplicar_feature_engineering_avanzado(df):
    """
    ACTUALIZADO:
    1. Se eliminan cálculos innecesarios (rango_colesterol, porcentaje_freq).
    2. Se calculan las variables compuestas necesarias antes de borrar las fuentes.
    """
    df = df.copy()

    # 1. Flag Depresión ST (Usada en Score Stress)
    if 'cambio_linea_corazon_ejercicio' in df.columns:
        df['flag_depresion_st'] = (df['cambio_linea_corazon_ejercicio'] > 0).astype(int)

    # 2. Flag Hipertensión (Absorbe info de tension_en_descanso)
    if 'tension_en_descanso' in df.columns:
        df['flag_hipertension'] = (df['tension_en_descanso'] > 130).astype(int)

    # NOTA: Eliminado calculo de 'porcentaje_frecuencia_max' (Grupo 2 - Redundancia negativa)
    # NOTA: Eliminado calculo de 'rango_colesterol' (Grupo 1 - Ruido)

    # 3. Score Respuesta al Estrés (Driver Tier 1)
    if 'dolor_pecho_con_ejercicio' in df.columns and 'flag_depresion_st' in df.columns:
        df['score_respuesta_stress'] = df['dolor_pecho_con_ejercicio'] + df['flag_depresion_st']

    # 4. Carga de Comorbilidad (Absorbe azucar y electro)
    cols_comorbilidad = ['azucar', 'flag_hipertension', 'electro_en_descanso']
    if set(cols_comorbilidad).issubset(df.columns):
        # Nos aseguramos que electro sea binario para la suma (0 = normal, 1,2 = anormal)
        electro_punto = (df['electro_en_descanso'] > 0).astype(int)
        df['carga_comorbilidad'] = df['azucar'] + df['flag_hipertension'] + electro_punto

    return df

In [14]:
def ejecutar_pruning_agresivo(df):
    """
    Elimina las variables de Grupo 1 (Ruido) y Grupo 2 (Redundancia/Absorbidas)
    después de haberlas utilizado para crear las variables compuestas.
    """
    df = df.copy()

    vars_a_eliminar = [
        # GRUPO 1: Ruido Puro
        'electro_en_descanso', # Usado en carga, adios.
        'colesterol',          # Inutil en este dataset, adios.
        # 'rango_colesterol' y 'thal_missing' ya no se generan.

        # GRUPO 2: Redundantes / Absorbidas
        'azucar',               # Absorbida en carga, adios.
        'tension_en_descanso'   # Absorbida en flag_hipertension, adios.
    ]

    # Eliminamos solo si existen (para evitar errores)
    cols_existentes = [c for c in vars_a_eliminar if c in df.columns]
    if cols_existentes:
        df = df.drop(columns=cols_existentes)

    return df

In [15]:
def optimizar_k_knn(X_train, k_range=[3, 5, 7, 9, 11, 15]):
    # Versión simplificada de la original
    X_temp = limpiar_ceros_fisiologicos(X_train)
    X_temp = clipear_outliers(X_temp)
    X_complete = X_temp.dropna().copy()
    if len(X_complete) < 50: return 5

    rmse_scores = {}
    scaler = RobustScaler()
    X_scaled_array = scaler.fit_transform(X_complete)
    np.random.seed(42)
    mask = np.random.rand(*X_scaled_array.shape) < 0.1
    X_missing_sim = X_scaled_array.copy()
    X_missing_sim[mask] = np.nan

    for k in k_range:
        imputer = KNNImputer(n_neighbors=k, weights='distance')
        X_imputed = imputer.fit_transform(X_missing_sim)
        error = np.sqrt(mean_squared_error(X_scaled_array[mask], X_imputed[mask]))
        rmse_scores[k] = error

    return min(rmse_scores, key=rmse_scores.get)

---
## APLICACIÓN DEL FLUJO DE PREPROCESADO DEFINIDO

---

In [16]:
df_train = df.copy()
df_test = test.copy()

df_train = limpieza_inicial(df_train)
df_test = limpieza_inicial(df_test)

target = "label"

X_train = df_train.drop(columns=target)
y_train = df_train[target]

if target in df_test.columns:
    X_test = df_test.drop(columns=target)
    y_test = df_test[target]
else:
    X_test = df_test.copy()

best_k = optimizar_k_knn(X_train)

# 3. Definición del Pipeline
pipeline_feature_engineering = Pipeline([
    ('limpieza_ceros', FunctionTransformer(limpiar_ceros_fisiologicos, validate=False)),
    ('clipear_outliers', FunctionTransformer(clipear_outliers, validate=False)),
    ('mnar_flags', FunctionTransformer(crear_flags_mnar, validate=False)),
    ('imputacion_robusta', RobustKNNImputerWrapper(n_neighbors=best_k)), # Usamos el K optimizado
    ('rounding', FunctionTransformer(redondear_imputaciones, validate=False)),
    ('feature_engineering', FunctionTransformer(aplicar_feature_engineering_avanzado, validate=False)),
    # NUEVO PASO: Pruning
    ('pruning_agresivo', FunctionTransformer(ejecutar_pruning_agresivo, validate=False)),
    ('final_scaler', RobustScaler())
]).set_output(transform="pandas")

---
## EJECUCIÓN Y VERIFICACIÓN DE TRANSFORMACIÓN

---

In [17]:
print("Ajustando pipeline con estrategia de Pruning Agresivo...")
X_train_prep = pipeline_feature_engineering.fit_transform(X_train)
y_train_prep = y_train.copy()

X_test_prep = pipeline_feature_engineering.transform(X_test)
print("\n--- Proceso finalizado ---")
print(f"Dimensiones Train final: {X_train_prep.shape}")
print(f"Columnas finales en el dataset: \n{X_train_prep.columns.tolist()}")

Ajustando pipeline con estrategia de Pruning Agresivo...

--- Proceso finalizado ---
Dimensiones Train final: (732, 14)
Columnas finales en el dataset: 
['edad', 'sexo', 'tipo_dolor_pecho', 'latidos_por_minuto', 'dolor_pecho_con_ejercicio', 'cambio_linea_corazon_ejercicio', 'forma_linea_corazon_ejercicio', 'num_venas_grandes', 'estado_corazon_thal', 'num_venas_grandes_is_missing', 'flag_depresion_st', 'flag_hipertension', 'score_respuesta_stress', 'carga_comorbilidad']


---
## BÚSQUEDA DE HIPERPARÁMETROS

---

---

SVM

---

In [18]:
print("--- Buscando la mejor configuración para SVM (Maximizando ACCURACY) ---")

svm_base = SVC(random_state=42)

# 2. Definimos la rejilla de hiperparámetros
# C: Controla la rigidez del margen.
#    - C bajo: Margen suave (más errores permitidos en train, mejor generalización).
#    - C alto: Margen duro (intenta clasificar todo bien en train).
# Gamma: Define cuánto influye un solo ejemplo.
#    - scale: Valor por defecto (suele funcionar bien).
#    - 0.1, 0.01: Valores manuales para probar sensibilidad.
param_grid = {
    'C': [0.1, 1, 10, 50],       # Probamos distintos niveles de regularización
    'kernel': ['rbf'],           # 'rbf' suele ser el mejor para datos complejos
    'gamma': ['scale', 0.1],     # Ajuste fino del kernel
    'class_weight': [None]       # IMPORTANTE: No balanceamos para priorizar Accuracy
}

grid_search_svm = GridSearchCV(
    svm_base,
    param_grid,
    cv=5,
    scoring='accuracy',  # Prioridad absoluta: Aciertos totales
    n_jobs=-1,           # Usar todos los núcleos del PC
    verbose=2            # Muestra progreso en tiempo real
)

grid_search_svm.fit(X_train_prep, y_train)

best_svm = grid_search_svm.best_estimator_

print(f"\nMejor Accuracy (SVM) conseguido en validación: {grid_search_svm.best_score_:.4f}")
print(f"Mejores parámetros SVM: {grid_search_svm.best_params_}")

--- Buscando la mejor configuración para SVM (Maximizando ACCURACY) ---
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Mejor Accuracy (SVM) conseguido en validación: 0.5205
Mejores parámetros SVM: {'C': 1, 'class_weight': None, 'gamma': 0.1, 'kernel': 'rbf'}


In [19]:
print("--- Afinando SVM (Zoom en torno a C=1 y gamma=0.1) ---")

# Exploramos valores cercanos a los ganadores anteriores
param_grid_svm_fine = {
    'C': [0.5, 0.8, 1, 1.2, 1.5, 2, 3],       # Alrededor de 1
    'gamma': [0.05, 0.08, 0.1, 0.12, 0.15],   # Alrededor de 0.1
    'kernel': ['rbf'],
    'class_weight': [None]
}

svm_fine = SVC(random_state=42)

grid_svm_fine = GridSearchCV(
    svm_fine,
    param_grid_svm_fine,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_svm_fine.fit(X_train_prep, y_train)

print(f"\nNuevo mejor Accuracy SVM: {grid_svm_fine.best_score_:.5f}")
print(f"Mejores parámetros SVM finos: {grid_svm_fine.best_params_}")

--- Afinando SVM (Zoom en torno a C=1 y gamma=0.1) ---
Fitting 5 folds for each of 35 candidates, totalling 175 fits

Nuevo mejor Accuracy SVM: 0.53006
Mejores parámetros SVM finos: {'C': 1, 'class_weight': None, 'gamma': 0.05, 'kernel': 'rbf'}


In [20]:
print("--- Probrar más combinaciones tras resultados positivos ---")

# 1. Definimos el modelo base
svm_base = SVC(kernel='rbf', probability=True, random_state=42)

# 2. Definimos la rejilla de búsqueda CENTRADA en los ganadores (C=1, gamma=0.05)
# Probamos valores ligeramente arriba y abajo para ver si el óptimo está cerca.
param_grid_svm_final = {
    'C': [0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5],
    'gamma': [0.03, 0.04, 0.05, 0.06, 0.07],
    'class_weight': [None]
}

# 3. Configuramos la búsqueda
grid_svm_final = GridSearchCV(
    svm_base,
    param_grid_svm_final,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# 4. Ejecutamos la búsqueda
print("Buscando la configuración perfecta alrededor de 0.565...")
grid_svm_final.fit(X_train_prep, y_train)

# 5. Resultados
best_svm_final = grid_svm_final.best_estimator_
print(f"\nMejor Accuracy (CV Local): {grid_svm_final.best_score_:.5f}")
print(f"Mejores parámetros: {grid_svm_final.best_params_}")

--- Probrar más combinaciones tras resultados positivos ---
Buscando la configuración perfecta alrededor de 0.565...
Fitting 5 folds for each of 40 candidates, totalling 200 fits

Mejor Accuracy (CV Local): 0.53007
Mejores parámetros: {'C': 1.1, 'class_weight': None, 'gamma': 0.04}


---
REGRESIÓN LOGÍSTICA

---

In [21]:
print("--- Buscando la mejor configuración para maximizar ACCURACY ---")

# 1. Definimos los parámetros a probar
# C: Inverso de la regularización.
#   - C alto: Se ajusta mucho a los datos de train (riesgo de overfitting, pero maximiza train).
#   - C bajo: Más generalista.
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'], # Probamos dos algoritmos matemáticos diferentes
    'class_weight': [None] # Forzamos a que NO balancee las clases
}

base_lr = LogisticRegression(random_state=42, max_iter=2000)

# 2. Configuramos la búsqueda priorizando 'accuracy'
grid_search = GridSearchCV(
    base_lr,
    param_grid,
    cv=5,
    scoring='accuracy',  # <--- Le decimos explícitamente: "quiero maximizar esto"
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_prep, y_train)

# 3. Resultados
best_model = grid_search.best_estimator_

print(f"\nMejor Accuracy conseguido en validación: {grid_search.best_score_:.4f}")
print(f"Mejores parámetros: {grid_search.best_params_}")

--- Buscando la mejor configuración para maximizar ACCURACY ---
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Mejor Accuracy conseguido en validación: 0.5314
Mejores parámetros: {'C': 0.1, 'class_weight': None, 'solver': 'lbfgs'}


In [22]:
print("--- Afinando Regresión Logística (Valores bajos de C) ---")

# valores más pequeños y granulares alrededor de 0.1
param_grid_fine = {
    'C': [0.01, 0.05, 0.08, 0.1, 0.15, 0.2],
    'solver': ['lbfgs', 'liblinear'],
    'class_weight': [None]
}

lr_fine = LogisticRegression(random_state=42, max_iter=2000)

grid_fine = GridSearchCV(
    lr_fine,
    param_grid_fine,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_fine.fit(X_train_prep, y_train)

print(f"\nNuevo mejor Accuracy: {grid_fine.best_score_:.5f}")
print(f"Mejores parámetros finos: {grid_fine.best_params_}")

--- Afinando Regresión Logística (Valores bajos de C) ---
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Nuevo mejor Accuracy: 0.53279
Mejores parámetros finos: {'C': 0.08, 'class_weight': None, 'solver': 'lbfgs'}
